# Student: Darren Chen
# ADS 507 
# Assignment 5.1  - SQL Query Performance Improvement

In [1]:
# kernel python 3.10.12 
# Install the mysql-connector-python package
# %pip install mysql
# %pip install mysql-connector-python

import mysql.connector 
import pandas as pd
import csv

Connect to Azure Database

In [ ]:
# Azure SQL Server Connection 
conn = mysql.connector.connect(
    host="mysqldchen.mysql.database.azure.com",
    user="dchenAdmin",
    port="3306",
    password="507password!",
    database="cdcyrbss"
)

cursor = conn.cursor()

In [ ]:
# run_query function 
def run_query(query):
    # try: (indent next for error handling)
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        print(row)
    # except mysql.connector.Error as err:
    #     print(f"Error: {err}")
    # finally:
    #     if conn.is_connected():
    #         cursor.close()
    #         conn.close()

Load ADSS data to sql datab

ETL DATA: Alcohol and Drug Services Study (ADSS), 1996-1999: [United States] (ICPSR 3088)
LINK: https://www.icpsr.umich.edu/web/NAHDAP/studies/3088/export
DOWNLOAD DELIMITED FILE


In [ ]:
# convert TSV to CSV

# Define the file paths 
tsv_file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0003\03088-0003-Data.tsv'
csv_file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0003\03088-0003-Data.csv'

# Open the TSV file for reading and the CSV file for writing
with open(tsv_file_path, 'r', newline='', encoding='utf-8') as tsv_file, \
     open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    
    # Create a TSV reader and a CSV writer
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    csv_writer = csv.writer(csv_file, delimiter=',')

    # Iterate over each row in the TSV file and write it to the CSV file
    for row in tsv_reader:
        csv_writer.writerow(row)

print(f"TSV file has been successfully converted to CSV and saved at: {csv_file_path}")

convert tsv file to csv for import to mysql database: 

in VSCode Terminal enter the following to convert to csv:  
>Import-Csv -Delimiter "`t" -Path "C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0004\03088-0004-Data.tsv" | Export-Csv -NoTypeInformation -Path "C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0004\03088-0004-Data.csv"

ICPSR_03088  Alcohol and Drug Services Study (ADSS)

Part 3 - Phase 2 - Main Incentive Abstract data table


In [9]:

# Define file path
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\data\\ICPSR_03088\\DS0003\\03088-0003-Data.csv"

# Load CSV into DataFrame
df_part3 = pd.read_csv(csv_file)

# Check if 'group' column exists before dropping
if 'group' in df_part3.columns:
    df_part3 = df_part3.drop(columns=['group'])
    print("Column 'group' has been removed.")
else:
    print("Column 'group' not found in the dataset.")

# Save updated DataFrame to a new CSV file
output_file = csv_file.replace(".csv", "_updated.csv")
df_part3.to_csv(output_file, index=False)

print(f"Updated file saved as: {output_file}")


Column 'group' has been removed.
Updated file saved as: C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0003\03088-0003-Data_updated.csv


In [10]:
# import mysql.connector
# import pandas as pd

# Database connection details
db_config = {
    "host": "mysqldchen.mysql.database.azure.com",
    "user": "dchenAdmin",
    "port": 3306,
    "password": "507password!",
    "database": "icpsr_03088"
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Generate table name
table_name = "adss_data_part3"

# Define file path
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\data\\ICPSR_03088\\DS0003\\03088-0003-Data_updated.csv"

# Load CSV into DataFrame
df_part3 = pd.read_csv(csv_file)

# Get column names and data types
columns = df_part3.columns
data_types = df_part3.dtypes

# Mapping pandas dtypes to MySQL column types
dtype_mapping = {
    "int64": "INT",
    "float64": "FLOAT",
    "object": "TEXT",
    "bool": "BOOLEAN"
}

# Define the table schema with caseid and facid as PRIMARY KEYS
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
for col, dtype in zip(columns, data_types):
    mysql_type = dtype_mapping.get(str(dtype), "TEXT")  # Default to TEXT if unknown
    create_table_query += f"`{col}` {mysql_type}, "

# Define composite primary key (caseid, facid)
create_table_query += "PRIMARY KEY (`caseid`, `facid`));"

# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_query = f"INSERT IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
# iteratively adds rows to the table
for _, row in df_part3.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully loaded into MySQL database.")

Data successfully loaded into MySQL database.


ICPSR_03088  Alcohol and Drug Services Study (ADSS)

Part 4 - Phase 2 - In-Treatment Methadone Abstract


In [14]:

# Define file path
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\data\\ICPSR_03088\\DS0004\\03088-0004-Data.csv"

# Load CSV into DataFrame
df_part4 = pd.read_csv(csv_file)

# Check if 'group' column exists before dropping
if 'group' in df_part4.columns:
    df_part4 = df_part4.drop(columns=['group'])
    print("Column 'group' has been removed.")
else:
    print("Column 'group' not found in the dataset.")

# Save updated DataFrame to a new CSV file
output_file = csv_file.replace(".csv", "_updated.csv")
df_part4.to_csv(output_file, index=False)

print(f"Updated file saved as: {output_file}")


Column 'group' has been removed.
Updated file saved as: C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0004\03088-0004-Data_updated.csv


In [15]:
# Database connection details
db_config = {
    "host": "mysqldchen.mysql.database.azure.com",
    "user": "dchenAdmin",
    "port": 3306,
    "password": "507password!",
    "database": "icpsr_03088"
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Generate table name
table_name = "adss_data_part4"

# Define file path
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\data\\ICPSR_03088\\DS0004\\03088-0004-Data_updated.csv"

# Load CSV into DataFrame
df_part4 = pd.read_csv(csv_file)

# Get column names and data types
columns = df_part4.columns
data_types = df_part4.dtypes

# Mapping pandas dtypes to MySQL column types
dtype_mapping = {
    "int64": "INT",
    "float64": "FLOAT",
    "object": "TEXT",
    "bool": "BOOLEAN"
}

# Define the table schema with caseid and facid as PRIMARY KEYS
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
for col, dtype in zip(columns, data_types):
    mysql_type = dtype_mapping.get(str(dtype), "TEXT")  # Default to TEXT if unknown
    create_table_query += f"`{col}` {mysql_type}, "

# Define composite primary key (caseid, facid)
create_table_query += "PRIMARY KEY (`caseid`, `facid`));"

# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_query = f"INSERT IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
# iteratively adds rows to the table
for _, row in df_part4.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully loaded into MySQL database.")

Data successfully loaded into MySQL database.


ICPSR_03088  Alcohol and Drug Services Study (ADSS)

Part 7 - Phase 3 - Follow-up Survey

In [16]:
# Load the data from tsv and remove column named 'group'
# Define file path
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\data\\ICPSR_03088\\DS0007\\03088-0007-Data.csv"

# Load CSV into DataFrame
df_part7 = pd.read_csv(csv_file)

# Check if 'group' column exists before dropping
if 'group' in df_part7.columns:
    df_part7 = df_part7.drop(columns=['group'])
    print("Column 'group' has been removed.")
else:
    print("Column 'group' not found in the dataset.")

# Save updated DataFrame to a new CSV file
output_file = csv_file.replace(".csv", "_updated.csv")
df_part7.to_csv(output_file, index=False)

print(f"Updated file saved as: {output_file}")


Column 'group' has been removed.
Updated file saved as: C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\ICPSR_03088\DS0007\03088-0007-Data_updated.csv


In [17]:
# Database connection details
db_config = {
    "host": "mysqldchen.mysql.database.azure.com",
    "user": "dchenAdmin",
    "port": 3306,
    "password": "507password!",
    "database": "icpsr_03088"
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Generate table name
table_name = "adss_data_part7"

# Define file path
csv_file = "C:\\Users\\darre\\OneDrive\\Documents\\ADS\\ADS 507 Data Engineering\\data\\ICPSR_03088\\DS0007\\03088-0007-Data_updated.csv"

# Load CSV into DataFrame
df_part7 = pd.read_csv(csv_file)

# Get column names and data types
columns = df_part7.columns
data_types = df_part7.dtypes

# Mapping pandas dtypes to MySQL column types
dtype_mapping = {
    "int64": "INT",
    "float64": "FLOAT",
    "object": "TEXT",
    "bool": "BOOLEAN"
}

# Define the table schema with caseid and facid as PRIMARY KEYS
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
for col, dtype in zip(columns, data_types):
    mysql_type = dtype_mapping.get(str(dtype), "TEXT")  # Default to TEXT if unknown
    create_table_query += f"`{col}` {mysql_type}, "

# Define composite primary key (caseid, facid)
create_table_query += "PRIMARY KEY (`caseid`, `facid`));"

# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_query = f"INSERT IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
# iteratively adds rows to the table
for _, row in df_part7.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully loaded into MySQL database.")

Data successfully loaded into MySQL database.


database: cdcyrbss   
CDC - Youth Risk Behavior Surveillance System (YRBSS) 

creating schema, and importing data 

primary key = 'CASEID'

In [ ]:
# 3ector
# import pandas as pd 

# Database connection details 
db_config = {
    "host": "mysqldchen.mysql.database.azure.com",
    "user": "dchenAdmin",
    "port": 3306,
    "password": "507password!",
    "database": "cdcyrbss"
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Load CSV file
csv_file = r"C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\CDC\XXHq.csv"

# Read CSV into pandas DataFrame
df = pd.read_csv(csv_file)

# Convert column names to lowercase and remove spaces for compatibility
df.columns = df.columns.str.lower().str.replace(" ", "_")

# Table name
table_name = "national2021"

# Get column names and data types
columns = df.columns
data_types = df.dtypes

# Mapping pandas dtypes to MySQL column types
dtype_mapping = {
    "int64": "INT",
    "float64": "FLOAT",
    "object": "TEXT",
    "bool": "BOOLEAN"
}

# Define the table schema dynamically
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
column_definitions = []

for col, dtype in zip(columns, data_types):
    mysql_type = dtype_mapping.get(str(dtype), "TEXT")  # Default to TEXT if unknown
    column_definitions.append(f"`{col}` {mysql_type}")

# Add primary key (caseid)
column_definitions.append("PRIMARY KEY (`record`)")
create_table_query += ", ".join(column_definitions) + ");"

# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_query = f"INSERT IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close the connection
conn.commit()
cursor.close()
conn.close()

print(" Data successfully loaded into MySQL database.")


C:\Users\darre\AppData\Local\Temp\ipykernel_39376\3460893445.py:21: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


 Data successfully loaded into MySQL database.


database: dawn   
Drug Abuse Warning Network (DAWN) 

creating schema, and importing data 

table name = 'data'

primary key = 'CASEID'

In [7]:
# convert DAWN data TSV to CSV

# Define the file paths 
tsv_file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\Drug Abuse Warning Network (DAWN) ICPSR_34565\DS0001\34565-0001-Data.tsv'
csv_file_path = r'C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\Drug Abuse Warning Network (DAWN) ICPSR_34565\DS0001\34565-0001-Data.csv'

# Open the TSV file for reading and the CSV file for writing
with open(tsv_file_path, 'r', newline='', encoding='utf-8') as tsv_file, \
     open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    
    # Create a TSV reader and a CSV writer
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    csv_writer = csv.writer(csv_file, delimiter=',')

    # Iterate over each row in the TSV file and write it to the CSV file
    for row in tsv_reader:
        csv_writer.writerow(row)

print(f"TSV file has been successfully converted to CSV and saved at: {csv_file_path}")

TSV file has been successfully converted to CSV and saved at: C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\Drug Abuse Warning Network (DAWN) ICPSR_34565\DS0001\34565-0001-Data.csv


In [5]:
import mysql.connector
import pandas as pd

# Database connection details
db_config = {
    "host": "mysqldchen.mysql.database.azure.com",
    "user": "dchenAdmin",
    "port": 3306,
    "password": "507password!",
    "database": "dawn"
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Load CSV file
csv_file = r"C:\Users\darre\OneDrive\Documents\ADS\ADS 507 Data Engineering\data\Drug Abuse Warning Network (DAWN) ICPSR_34565\DS0001\34565-0001-Data.csv"

# Read only first 5000 rows from CSV
df = pd.read_csv(csv_file)

# Adjust column names
df.columns = df.columns.str.lower().str.replace(" ", "_")

table_name = "data"
columns = df.columns
data_types = df.dtypes

dtype_mapping = {
    "int64": "INT",
    "float64": "FLOAT",
    "object": "TEXT",
    "bool": "BOOLEAN"
}

# Define the table schema
column_definitions = [
    f"`{col}` {dtype_mapping.get(str(dtype), 'TEXT')} {'PRIMARY KEY' if col == 'caseid' else ''}"
    for col, dtype in zip(columns, data_types)
]

# Create the partitioned table
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    {', '.join(column_definitions)}
) PARTITION BY HASH(caseid) PARTITIONS 20;
"""

cursor.execute(create_table_query)
conn.commit()
print("Table created successfully.")

# Prepare insert query
insert_query = f"""
INSERT IGNORE INTO {table_name} ({', '.join(columns)})
VALUES ({', '.join(['%s'] * len(columns))})
"""

data = df.where(pd.notnull(df), None).values.tolist()

cursor.executemany(insert_query, data)
conn.commit()
print("records inserted.")

cursor.close()
conn.close()


KeyError: 'record'

#  Loading Data to Unified Database

In [6]:
import mysql.connector

db_config = {
    'host': 'mysqldchen.mysql.database.azure.com',
    'user': 'dchenAdmin',
    'password': '507password!',
    'port': 3306
}

# Connect to MySQL server
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create new database 'analytics_db'
cursor.execute("CREATE DATABASE IF NOT EXISTS analytics_db;")

print("✅ Created new database: analytics_db")

cursor.close()
conn.close()

✅ Created new database: analytics_db


In [ ]:
import pandas as pd

# Load datasets
dawn_df = pd.read_csv("dawn_data.tsv", sep="\t")
yrbss_df = pd.read_csv("yrbss_data.csv")
nahdap_df = pd.read_csv("nahdap_data.csv")


In [ ]:
dawn_df.fillna("Unknown", inplace=True)
yrbss_df.fillna("Unknown", inplace=True)
nahdap_df.fillna(0, inplace=True)


In [ ]:
yrbss_df["substance_abuse_rate"] = yrbss_df["students_using_drugs"] / yrbss_df["total_students"] * 100


In [ ]:
run_query("""
CREATE TABLE dawn (
    case_id INT PRIMARY KEY,
    drug_name VARCHAR(255),
    incident_type VARCHAR(255),
    age_group VARCHAR(50),
    location VARCHAR(255),
    year INT
);

CREATE TABLE yrbss (
    survey_id INT PRIMARY KEY,
    state VARCHAR(255),
    total_students INT,
    students_using_drugs INT,
    substance_abuse_rate FLOAT
);

CREATE TABLE nahdap (
    record_id INT PRIMARY KEY,
    facility_name VARCHAR(255),
    treatment_capacity INT,
    patients_served INT
);
"""
)

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host="mysqldchen.mysql.database.azure.com",
    user="dchenAdmin",
    password="507password!",
    database="icpsr_03088"
)
cursor = conn.cursor()

# Example: Insert DAWN Data
for _, row in dawn_df.iterrows():
    cursor.execute("INSERT INTO dawn (case_id, drug_name, incident_type, age_group, location, year) VALUES (%s, %s, %s, %s, %s, %s)", 
                   (row['case_id'], row['drug_name'], row['incident_type'], row['age_group'], row['location'], row['year']))

conn.commit()
cursor.close()
conn.close()


# Automation and Scheduling 

In [ ]:
# Install the apache-airflow package
# %pip install apache-airflow

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

In [ ]:

def extract():
    print("Extracting data...")

def transform():
    print("Transforming data...")

def load():
    print("Loading data into database...")

dag = DAG(
    'data_pipeline',
    schedule_interval='@daily',
    start_date=datetime(2024, 1, 1),
)

task_extract = PythonOperator(task_id='extract', python_callable=extract, dag=dag)
task_transform = PythonOperator(task_id='transform', python_callable=transform, dag=dag)
task_load = PythonOperator(task_id='load', python_callable=load, dag=dag)

task_extract >> task_transform >> task_load


# Automated ETL pipline using Airflow